# RawデータからYOLO形式データセット生成＆S3パッキング

## このノートブックの目的

Kaggleから取得したrawデータ（zip）を `/content/tmp` 上で展開し、
rawラベル（LaTeX＋座標）と画像を突合して、
指定したクラス（例：1〜9、+,-,*,/,=,(,),x など）のみ抽出し、
YOLO学習用に以下の構造を生成します：

```
out/
  train/images
  train/labels
  val/images
  val/labels
  mapping.json
  data.yaml
  manifest.json
```

最後に tar にまとめてS3へアップできる状態にします。

## 入力

- Kaggleから落とした raw.zip（または展開済みフォルダ）
- raw画像フォルダ（例：raw/images/... または batch*/background_images/...）
- rawラベル（JSON形式、LaTeX文字列＋bbox）
- 既存の mapping.json がある場合はそれも入力として使える（必須ではない）

## 出力

### YOLOデータ（展開状態）
`/content/tmp/yolo_out/` 配下に以下を生成：
- `train/images/`, `train/labels/`
- `val/images/`, `val/labels/`
- `mapping.json`, `data.yaml`, `manifest.json`

### tarアーカイブ（S3保管用）
- `train_images.tar`, `train_labels.tar`
- `val_images.tar`, `val_labels.tar`
- `meta.tar`（mapping.json, data.yaml, manifest.json をまとめる）

## 実行順序

**上から順に全てのセルを実行してください。**

各セルは冪等性（idempotent）を保証しており、
既に処理済みの場合はスキップされます。


##セル0: awsの認証
下のコードの"AWS_ACCESS_KEY_ID"と"AWS_SECRET_ACCESS_KEY"を埋めて実行してから下のコードを実行する

In [ ]:
import os

os.environ["AWS_ACCESS_KEY_ID"] = "AKIAxxxxxxxxxxxxxxxx"#埋めてね！！！！！！！！
os.environ["AWS_SECRET_ACCESS_KEY"] = "xxxxxxxxxxxxxxxxxxxx"#埋めてね！！！！！！！！
os.environ["AWS_DEFAULT_REGION"] = "ap-northeast-1"

print("AWS env set")

assert "AWS_ACCESS_KEY_ID" in os.environ, "AWS_ACCESS_KEY_ID not set"
assert "AWS_SECRET_ACCESS_KEY" in os.environ, "AWS_SECRET_ACCESS_KEY not set"


In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = 'masapirika'
os.environ['KAGGLE_KEY'] = '================'

## セル1: 設定（ここだけ触ればOK）


In [ ]:
# ============================================
# 設定（ここだけ変更すればOK）
# ============================================

import os
from pathlib import Path

# ===== パス設定 =====
# Kaggleから落としたraw.zipのパス（既にDL済みならそのパス、未DLならNone）
RAW_ZIP_PATH = None  # 例: "/content/raw.zip" または None（セル3でDL）

# rawデータの展開先（/content/tmp/raw）
RAW_DIR = Path("/content/tmp/raw")

# YOLOデータの出力先
OUT_DIR = Path("/content/tmp/yolo_out")

# ===== train/val分割設定 =====
SEED = 42  # 再現性のためのシード
VAL_RATIO = 0.2  # 20%をvalに

# ===== 抽出対象クラス一覧 =====
# 拡張版：約61クラス（Kaggle Aida Calculus Math Handwriting Recognition Dataset対応）
# カスタマイズする場合はここを変更
TARGET_CLASSES = [
    # 数字（10クラス）
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    # 基本演算子（5クラス）
    '+', '-', '*', '/', '=',
    # アルファベット変数（小文字26クラス）
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
    'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
    # 括弧類（3クラス）
    '(', ')', '|',  # 丸括弧2種 + 絶対値記号
    # ギリシャ文字・定数（3クラス）
    'π',  # 円周率
    'θ',  # 角度変数
    '∞',  # 無限大
    # 注意: 'e'（自然対数の底）はアルファベット変数の'e'と重複するため削除
    # 特殊記号（2クラス）
    '→',  # 矢印（リミットの「～に近づく」）
    '√',  # ルート記号（平方根）
    # 関数名（12クラス）
    'sin',  # 正弦
    'cos',  # 余弦
    'tan',  # 正接
    'sec',  # 正割
    'csc',  # 余割
    'cot',  # 余接
    'ln',   # 自然対数
    'log',  # 対数
    'exp',  # 指数関数
    'sqrt', # 平方根
    'lim',  # 極限
    'abs',  # 絶対値
]

# ===== S3設定 =====
S3_BUCKET = "km62m-ml-storage"  # S3バケット名
S3_PREFIX = "yolo-dataset/v1"  # S3プレフィックス
UPLOAD_TO_S3 = True  # TrueにするとS3へアップロード

# ===== Kaggle設定（セル3で使用） =====
KAGGLE_DATASET = "aidapearson/ocr-data"  # Kaggleデータセット名
KAGGLE_DOWNLOAD_DIR = Path("/content/tmp")

# ===== その他 =====
# 既存のOUT_DIRを削除するか（True: 削除、False: 上書き）
CLEAN_OUT_DIR = False  # 既存データを保持したい場合はFalse

print("✓ 設定完了")
print(f"  RAW_DIR: {RAW_DIR}")
print(f"  OUT_DIR: {OUT_DIR}")
print(f"  VAL_RATIO: {VAL_RATIO}")
print(f"  対象クラス数: {len(TARGET_CLASSES)}")


## セル5.5: 任意実行 - 生データの形式確認（lim、abs、exp、sqrtなど）

このセルは任意実行です。lim、abs、exp、sqrtなどの関数名が実際のデータでどのように表現されているかを確認できます。


In [ ]:
# ============================================
# 任意実行: 生データの形式確認（lim、abs、exp、sqrtなど）
# ============================================
# このセルは任意実行です。必要に応じて実行してください。

from collections import defaultdict

# 確認対象の関数名
target_functions = ['lim', 'abs', 'exp', 'sqrt']

# 関数名の出現パターンを記録
function_patterns = defaultdict(list)
latex_command_patterns = defaultdict(list)

# 確認するサンプル数（必要に応じて変更）
max_samples_to_check = 1000

print("=" * 80)
print("関数名の出現パターン確認")
print("=" * 80)
print(f"確認するサンプル数: {max_samples_to_check}")
print()

sample_count = 0
for uuid, sample in samples.items():
    if sample_count >= max_samples_to_check:
        break
    
    latex_chars = sample.get('latex_chars', [])
    
    # 1. 連続する文字列として関数名を探す（'l', 'i', 'm' → 'lim'）
    for func in target_functions:
        for i in range(len(latex_chars) - len(func) + 1):
            candidate = ''.join(latex_chars[i:i+len(func)])
            if candidate == func:
                context = latex_chars[max(0, i-3):min(len(latex_chars), i+len(func)+3)]
                function_patterns[func].append({
                    'context': context,
                    'uuid': uuid,
                    'type': 'concatenated'
                })
    
    # 2. LaTeXコマンド形式を探す（'\lim_', '\sqrt'など）
    for func in target_functions:
        # LaTeXコマンドのパターン
        latex_patterns = [
            f'\\{func}',      # '\lim', '\sqrt'
            f'\\{func}_',     # '\lim_', '\sqrt_'
            f'\\{func}{{',    # '\lim{', '\sqrt{'
        ]
        
        for char in latex_chars:
            for pattern in latex_patterns:
                if pattern in char:
                    latex_command_patterns[func].append({
                        'char': char,
                        'uuid': uuid,
                        'type': 'latex_command'
                    })
                    break
    
    sample_count += 1

# 結果を表示
print(f"確認したサンプル数: {sample_count}")
print()

for func in target_functions:
    print(f"\n{func}:")
    
    # 連続文字列としての出現
    if func in function_patterns:
        print(f"  連続文字列としての出現: {len(function_patterns[func])} 回")
        if len(function_patterns[func]) > 0:
            print("  例（最初の3件）:")
            for i, pattern in enumerate(function_patterns[func][:3]):
                print(f"    {i+1}. コンテキスト: {pattern['context']}")
    else:
        print(f"  連続文字列としての出現: 0 回")
    
    # LaTeXコマンド形式での出現
    if func in latex_command_patterns:
        print(f"  LaTeXコマンド形式での出現: {len(latex_command_patterns[func])} 回")
        if len(latex_command_patterns[func]) > 0:
            print("  例（最初の5件）:")
            unique_chars = set()
            for pattern in latex_command_patterns[func][:10]:
                unique_chars.add(pattern['char'])
            for i, char in enumerate(list(unique_chars)[:5]):
                print(f"    {i+1}. '{char}'")
    else:
        print(f"  LaTeXコマンド形式での出現: 0 回")
    
    if func not in function_patterns and func not in latex_command_patterns:
        print(f"  → この関数名はデータ内で見つかりませんでした")

# すべてのvisible_charsのユニークな値（関数名関連）を確認
print("\n" + "=" * 80)
print("visible_charsのユニークな値（関数名関連・長さ2文字以上）")
print("=" * 80)

unique_chars = set()
for uuid, sample in list(samples.items())[:max_samples_to_check]:
    latex_chars = sample.get('latex_chars', [])
    for char in latex_chars:
        if len(char) > 1 or any(func in char for func in target_functions):
            unique_chars.add(char)

print(f"ユニークな値の数: {len(unique_chars)}")
if len(unique_chars) > 0:
    print("値の例（最初の30件）:")
    for char in sorted(unique_chars)[:30]:
        print(f"  '{char}'")
else:
    print("  長さ2文字以上の値は見つかりませんでした")

print("\n✓ 確認完了")


## セル5.6: 任意実行 - JSONデータの構造確認

このセルは任意実行です。JSONファイルの実際の構造を確認できます。


In [ ]:
# ============================================
# 任意実行: JSONデータの構造確認
# ============================================
# このセルは任意実行です。JSONファイルの構造を確認できます。

import json
from pathlib import Path

# JSONファイルを検索
json_files = list(RAW_DIR.rglob("kaggle_data_*.json"))
if len(json_files) == 0:
    print("❌ JSONファイルが見つかりません。セル4（unzip）を実行してください。")
else:
    print(f"見つかったJSONファイル: {len(json_files)} 件")
    print(f"最初のファイルを確認: {json_files[0]}")
    print()
    
    # 最初のJSONファイルを読み込む
    with open(json_files[0], 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    if isinstance(data, list) and len(data) > 0:
        sample = data[0]
        print("=" * 80)
        print("サンプルデータの構造（最初の1件）")
        print("=" * 80)
        print(f"UUID: {sample.get('uuid', 'N/A')}")
        print(f"LaTeX: {sample.get('latex', 'N/A')}")
        print()
        
        if 'image_data' in sample:
            img_data = sample['image_data']
            print("image_dataの構造:")
            print(f"  width: {img_data.get('width', 'N/A')}")
            print(f"  height: {img_data.get('height', 'N/A')}")
            
            visible_chars = img_data.get('visible_latex_chars', [])
            full_chars = img_data.get('full_latex_chars', [])
            
            print(f"\n  visible_latex_charsの数: {len(visible_chars)}")
            print(f"  full_latex_charsの数: {len(full_chars)}")
            
            print(f"\n  visible_latex_chars（最初の20件）:")
            for i, char in enumerate(visible_chars[:20]):
                print(f"    [{i}] '{char}'")
            
            if len(visible_chars) > 20:
                print(f"    ... 他 {len(visible_chars) - 20} 件")
            
            print(f"\n  full_latex_chars（最初の20件）:")
            for i, char in enumerate(full_chars[:20]):
                print(f"    [{i}] '{char}'")
            
            if len(full_chars) > 20:
                print(f"    ... 他 {len(full_chars) - 20} 件")
            
            # lim、abs、exp、sqrtを含む文字を探す
            print(f"\n  lim/abs/exp/sqrtを含む文字:")
            target_patterns = ['lim', 'abs', 'exp', 'sqrt']
            found_chars = []
            for char in visible_chars:
                for pattern in target_patterns:
                    if pattern in char or f'\\{pattern}' in char:
                        found_chars.append(char)
                        break
            
            if found_chars:
                unique_found = list(set(found_chars))
                for char in unique_found[:10]:
                    print(f"    '{char}'")
                if len(unique_found) > 10:
                    print(f"    ... 他 {len(unique_found) - 10} 件")
            else:
                print("    見つかりませんでした")
    else:
        print("❌ データが空です。")
    
    print("\n✓ 確認完了")


## セル5.7: 任意実行 - データセット内の記号の網羅性確認

このセルは任意実行です。データセットに含まれているすべての記号が認識できているかを確認できます。


In [ ]:
# ============================================
# 任意実行: データセット内の記号の網羅性確認
# ============================================
# このセルは任意実行です。データセットに含まれているすべての記号が認識できているかを確認できます。

from collections import defaultdict

# 確認するサンプル数（必要に応じて変更）
max_samples_to_check = 10000

print("=" * 80)
print("データセット内の記号の網羅性確認")
print("=" * 80)
print(f"確認するサンプル数: {max_samples_to_check}")
print()

# データセット内のすべてのユニークな記号を収集
all_unique_chars = set()
sample_count = 0

for uuid, sample in samples.items():
    if sample_count >= max_samples_to_check:
        break
    
    latex_chars = sample.get('latex_chars', [])
    for char in latex_chars:
        all_unique_chars.add(char)
    
    sample_count += 1

print(f"確認したサンプル数: {sample_count}")
print(f"データセット内のユニークな記号数: {len(all_unique_chars)}")
print()

# 各記号が認識できるかチェック
recognized_chars = []
unrecognized_chars = []
recognized_to_class = {}

for char in sorted(all_unique_chars):
    class_char = latex_to_class(char)
    if class_char is not None:
        recognized_chars.append(char)
        recognized_to_class[char] = class_char
    else:
        unrecognized_chars.append(char)

print("=" * 80)
print("認識結果")
print("=" * 80)
print(f"認識できた記号: {len(recognized_chars)} / {len(all_unique_chars)} ({len(recognized_chars)/len(all_unique_chars)*100:.1f}%)")
print(f"認識できなかった記号: {len(unrecognized_chars)} / {len(all_unique_chars)} ({len(unrecognized_chars)/len(all_unique_chars)*100:.1f}%)")
print()

# 認識できなかった記号を表示
if len(unrecognized_chars) > 0:
    print("=" * 80)
    print("⚠️  認識できなかった記号（要確認）")
    print("=" * 80)
    
    # 出現頻度も確認
    char_counts = defaultdict(int)
    for uuid, sample in list(samples.items())[:max_samples_to_check]:
        latex_chars = sample.get('latex_chars', [])
        for char in latex_chars:
            if char in unrecognized_chars:
                char_counts[char] += 1
    
    # 出現頻度順にソート
    sorted_unrecognized = sorted(char_counts.items(), key=lambda x: x[1], reverse=True)
    
    print(f"\n出現頻度順（上位20件）:")
    for char, count in sorted_unrecognized[:20]:
        print(f"  '{char}': {count} 回")
    
    if len(sorted_unrecognized) > 20:
        print(f"  ... 他 {len(sorted_unrecognized) - 20} 件")
else:
    print("✓ すべての記号が認識できています！")

# 認識できた記号のマッピング（一部表示）
print("\n" + "=" * 80)
print("認識できた記号のマッピング例（最初の30件）")
print("=" * 80)
for i, char in enumerate(sorted(recognized_chars)[:30]):
    print(f"  '{char}' → '{recognized_to_class[char]}'")

if len(recognized_chars) > 30:
    print(f"  ... 他 {len(recognized_chars) - 30} 件")

# TARGET_CLASSESに含まれているかも確認
print("\n" + "=" * 80)
print("TARGET_CLASSESとの対応確認")
print("=" * 80)

target_class_set = set(TARGET_CLASSES)
mapped_to_target = []
mapped_to_non_target = []

for char, class_char in recognized_to_class.items():
    if class_char in target_class_set:
        mapped_to_target.append((char, class_char))
    else:
        mapped_to_non_target.append((char, class_char))

print(f"TARGET_CLASSESにマッピングされた記号: {len(mapped_to_target)} 件")
print(f"TARGET_CLASSES以外にマッピングされた記号: {len(mapped_to_non_target)} 件")

if len(mapped_to_non_target) > 0:
    print("\n⚠️  TARGET_CLASSES以外にマッピングされた記号:")
    for char, class_char in mapped_to_non_target[:10]:
        print(f"  '{char}' → '{class_char}' (TARGET_CLASSESに含まれていません)")
    if len(mapped_to_non_target) > 10:
        print(f"  ... 他 {len(mapped_to_non_target) - 10} 件")

print("\n✓ 確認完了")


## セル2: 依存インストール


In [ ]:
# ============================================
# 依存パッケージのインストール
# ============================================

import subprocess
import sys

def install_package(package, import_name=None):
    """
    パッケージをインストール（必要に応じて）
    package: pipでインストールするパッケージ名
    import_name: import時に使用する名前（Noneの場合はpackageと同じ）
    """
    if import_name is None:
        import_name = package
    
    try:
        __import__(import_name)
        print(f"✓ {package} は既にインストール済み")
    except ImportError:
        print(f"📦 {package} をインストール中...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        print(f"✓ {package} インストール完了")

# 必要なパッケージ
# (package_name, import_name) のタプル形式
# import_nameがNoneの場合はpackage_nameと同じ
packages = [
    ("kaggle", None),
    ("pyarrow", None),
    ("pandas", None),
    ("tqdm", None),
    ("Pillow", "PIL"),  # Pillowはpip名だが、import時はPIL
    ("pyyaml", None),
]

for pkg_info in packages:
    if isinstance(pkg_info, tuple):
        pkg, import_name = pkg_info
    else:
        pkg, import_name = pkg_info, None
    install_package(pkg, import_name)

# AWS CLI（S3アップロード用）
try:
    result = subprocess.run(["aws", "--version"], capture_output=True, text=True)
    print(f"✓ AWS CLI は既にインストール済み: {result.stdout.strip()}")
except FileNotFoundError:
    print("📦 AWS CLI をインストール中...")
    subprocess.check_call(["pip", "install", "-q", "awscli"])
    print("✓ AWS CLI インストール完了")

print("\n✓ 全ての依存パッケージの準備完了")


## セル3: Kaggleからraw.zip取得（オプション）


In [ ]:
import os
import subprocess
from pathlib import Path

# =========================
# Kaggle認証の確認
# =========================
has_env = bool(os.getenv("KAGGLE_USERNAME")) and bool(os.getenv("KAGGLE_KEY"))
has_json = (Path.home() / ".config" / "kaggle" / "kaggle.json").exists() or (Path.home() / ".kaggle" / "kaggle.json").exists()

if not (has_env or has_json):
    print("⚠️ Kaggle認証が見つかりません")
    print("  - 環境変数 KAGGLE_USERNAME / KAGGLE_KEY を設定するか")
    print("  - ~/.config/kaggle/kaggle.json または ~/.kaggle/kaggle.json を配置してください")
else:
    print("✓ Kaggle認証OK（環境変数 or kaggle.json）")

# =========================
# KaggleからDL（必要なら）- CLI使用（RAM死を避けるため）
# =========================
if RAW_ZIP_PATH is None or not Path(RAW_ZIP_PATH).exists():
    print(f"\n📥 KaggleからDL中: {KAGGLE_DATASET}")
    print("  CLIコマンドを使用（Python APIより安定）")
    KAGGLE_DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)

    # kaggle CLIコマンドでダウンロード
    subprocess.run(
        ["kaggle", "datasets", "download", "-d", KAGGLE_DATASET, "-p", str(KAGGLE_DOWNLOAD_DIR)],
        check=True
    )

    zip_files = sorted(KAGGLE_DOWNLOAD_DIR.glob("*.zip"), key=lambda p: p.stat().st_mtime, reverse=True)
    if zip_files:
        RAW_ZIP_PATH = str(zip_files[0])
        size_gb = zip_files[0].stat().st_size / (1024 ** 3)
        print(f"✓ DL完了: {RAW_ZIP_PATH}")
        print(f"  サイズ: {size_gb:.2f} GB")
    else:
        raise FileNotFoundError("zipファイルが見つかりませんでした（DL先を確認して）")
else:
    print(f"\n✓ 既存zipを使用: {RAW_ZIP_PATH}")

print(f"\n最終RAW_ZIP_PATH: {RAW_ZIP_PATH}")


## セル4: unzip（/content/tmpへ）


In [ ]:
# ============================================
# unzip（/content/tmpへ展開）
# ============================================

import zipfile
from tqdm import tqdm

if RAW_ZIP_PATH is None or not Path(RAW_ZIP_PATH).exists():
    raise FileNotFoundError(f"RAW_ZIP_PATHが見つかりません: {RAW_ZIP_PATH}")

# 展開済みかチェック
if RAW_DIR.exists() and any(RAW_DIR.iterdir()):
    print(f"✓ 既に展開済み: {RAW_DIR}")
    print(f"  スキップします")
else:
    print(f"📦 zipファイルを展開中...")
    print(f"  ソース: {RAW_ZIP_PATH}")
    print(f"  展開先: {RAW_DIR}")
    
    RAW_DIR.mkdir(parents=True, exist_ok=True)
    
    # zipファイルを開く
    with zipfile.ZipFile(RAW_ZIP_PATH, 'r') as zip_ref:
        # ファイルリストを取得
        file_list = zip_ref.namelist()
        
        # 進捗バー付きで展開
        for file_info in tqdm(file_list, desc="展開中", unit="ファイル"):
            zip_ref.extract(file_info, RAW_DIR)
    
    print(f"\n✓ 展開完了: {RAW_DIR}")
    
    # 展開結果の確認
    json_files = list(RAW_DIR.rglob("kaggle_data_*.json"))
    print(f"  見つかったJSONファイル: {len(json_files)} 件")
    
    # background_imagesディレクトリを探す
    bg_dirs = list(RAW_DIR.rglob("background_images"))
    print(f"  見つかったbackground_imagesディレクトリ: {len(bg_dirs)} 件")


## セル5: rawのインデックス作成


In [ ]:
# ============================================
# rawのインデックス作成
# ============================================

import json
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm
import sys

# リポジトリのパスを追加（configをインポートするため）
repo_path = Path("/content/last_assignment_progzissen")
if not repo_path.exists():
    # Gitリポジトリをクローン（まだの場合）
    import subprocess
    print("📦 Gitリポジトリをクローン中...")
    subprocess.run(["git", "clone", "-b", "feature/onnx_recognizer", 
                    "https://github.com/masararutaru/last_assignment_progzissen.git", 
                    str(repo_path)], check=True)
    print("✓ クローン完了")

sys.path.insert(0, str(repo_path / "python" / "data"))
from config import CLASSES, CLASS_TO_ID, latex_to_class

print("=" * 80)
print("rawデータのインデックス作成")
print("=" * 80)

# JSONファイルを検索
json_files = list(RAW_DIR.rglob("kaggle_data_*.json"))
print(f"\n見つかったJSONファイル: {len(json_files)} 件")

if len(json_files) == 0:
    raise FileNotFoundError(f"JSONファイルが見つかりません: {RAW_DIR}")

# データ構造を構築
# samples: {uuid: {'latex_chars': [...], 'bboxes': [...], 'image_path': Path}}
samples = {}
image_path_cache = {}  # {uuid: Path}

# 各JSONファイルを処理
print("\nJSONファイルを読み込み中...")
for json_path in tqdm(json_files, desc="JSON処理"):
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        if not isinstance(data, list):
            continue
        
        # JSONファイルの親ディレクトリからbackground_imagesを探す
        json_dir = json_path.parent
        parent_dir = json_dir.parent
        background_images_dir = parent_dir / "background_images"
        
        # 画像ファイルをキャッシュ（このbatch内）
        if background_images_dir.exists():
            for img_file in background_images_dir.glob("*"):
                if img_file.is_file():
                    uuid_key = img_file.stem
                    if uuid_key not in image_path_cache:
                        image_path_cache[uuid_key] = img_file
        
        # 各サンプルを処理
        for sample in data:
            if not isinstance(sample, dict):
                continue
            
            uuid = sample.get('uuid', '')
            if not uuid:
                continue
            
            if 'image_data' not in sample:
                continue
            
            img_data = sample['image_data']
            visible_chars = img_data.get('visible_latex_chars', [])
            xmins = img_data.get('xmins', [])
            xmaxs = img_data.get('xmaxs', [])
            ymins = img_data.get('ymins', [])
            ymaxs = img_data.get('ymaxs', [])
            
            # bboxの数を確認
            if len(visible_chars) != len(xmins) or len(visible_chars) != len(xmaxs) or \
               len(visible_chars) != len(ymins) or len(visible_chars) != len(ymaxs):
                continue
            
            # 画像パスを検索
            image_path = image_path_cache.get(uuid)
            if image_path is None:
                # キャッシュにない場合は直接検索
                if background_images_dir.exists():
                    for ext in ['.png', '.jpg', '.jpeg']:
                        test_path = background_images_dir / f"{uuid}{ext}"
                        if test_path.exists():
                            image_path = test_path
                            image_path_cache[uuid] = image_path
                            break
            
            # サンプルを保存
            samples[uuid] = {
                'latex_chars': visible_chars,
                'bboxes': list(zip(xmins, ymins, xmaxs, ymaxs)),  # (xmin, ymin, xmax, ymax)
                'image_path': image_path
            }
    
    except Exception as e:
        print(f"\n⚠️  警告: {json_path} の処理に失敗: {e}")
        continue

print(f"\n✓ インデックス作成完了")
print(f"  総サンプル数: {len(samples)} 件")
print(f"  画像が見つかったサンプル: {sum(1 for s in samples.values() if s['image_path'] is not None)} 件")
print(f"  画像が見つからないサンプル: {sum(1 for s in samples.values() if s['image_path'] is None)} 件")


## セル6: クラス抽出・正規化


In [ ]:
# ============================================
# クラス抽出・正規化
# ============================================

from collections import defaultdict

print("=" * 80)
print("クラス抽出・正規化")
print("=" * 80)

# 対象クラスのIDマッピングを作成（0始まり連番に再マッピング）
target_class_to_new_id = {cls: idx for idx, cls in enumerate(TARGET_CLASSES)}
print(f"\n対象クラス数: {len(TARGET_CLASSES)}")
print(f"  クラス: {', '.join(TARGET_CLASSES)}")

# 抽出・変換後のデータ
# processed_samples: {uuid: {'class_ids': [...], 'bboxes_yolo': [...], 'image_path': Path}}
processed_samples = {}

stats = defaultdict(int)  # クラス別統計
skipped_no_target = 0  # 対象クラスがないサンプル
skipped_invalid_bbox = 0  # 無効なbbox
skipped_no_image = 0  # 画像が見つからない

print("\nサンプルを処理中...")
for uuid, sample in tqdm(samples.items(), desc="クラス抽出"):
    # 画像パスの確認
    image_path = sample['image_path']
    if image_path is None or not image_path.exists():
        skipped_no_image += 1
        continue
    
    latex_chars = sample['latex_chars']
    bboxes = sample['bboxes']
    
    # 対象クラスのみ抽出
    class_ids = []
    bboxes_yolo = []
    
    # 関数名リスト
    function_names = ['sin', 'cos', 'tan', 'sec', 'csc', 'cot', 'ln', 'log', 'exp', 'sqrt', 'lim', 'abs']
    
    i = 0
    while i < len(latex_chars):
        # まず、LaTeXコマンド形式をチェック（'\lim_', '\sqrt'など）
        latex_char = latex_chars[i]
        class_char_from_latex = latex_to_class(latex_char)
        
        # LaTeXコマンドが関数名として認識された場合
        if class_char_from_latex in function_names:
            matched_function = class_char_from_latex
            matched_length = 1  # LaTeXコマンドは1つの文字列として保存されている
        else:
            # 関数名のマッチングを試行（連続する文字列: 'l', 'i', 'm' → 'lim'）
            matched_function = None
            matched_length = 0
            
            for func_name in function_names:
                if i + len(func_name) <= len(latex_chars):
                    # 連続する文字列が関数名と一致するかチェック
                    candidate = ''.join(latex_chars[i:i+len(func_name)])
                    if candidate == func_name:
                        matched_function = func_name
                        matched_length = len(func_name)
                        break
        
        if matched_function:
            # 関数名として処理
            start_idx = i
            end_idx = i + matched_length - 1
            
            if end_idx < len(bboxes):
                # 関数全体のbboxを計算
                xmin = min(bboxes[j][0] for j in range(start_idx, end_idx+1))
                ymin = min(bboxes[j][1] for j in range(start_idx, end_idx+1))
                xmax = max(bboxes[j][2] for j in range(start_idx, end_idx+1))
                ymax = max(bboxes[j][3] for j in range(start_idx, end_idx+1))
                
                # 新しいクラスIDを取得
                new_class_id = target_class_to_new_id[matched_function]
                class_char = matched_function
            else:
                i += matched_length
                continue
        else:
            # 通常の文字として処理
            latex_char = latex_chars[i]
            class_char = latex_to_class(latex_char)
            
            if class_char is None or class_char not in TARGET_CLASSES:
                i += 1
                continue
            
            # 新しいクラスIDを取得（0始まり連番）
            new_class_id = target_class_to_new_id[class_char]
            
            # bboxを取得
            xmin, ymin, xmax, ymax = bboxes[i]
        
        # 無効なbboxをチェック
        if xmin >= xmax or ymin >= ymax:
            skipped_invalid_bbox += 1
            if matched_function:
                i += matched_length
            else:
                i += 1
            continue
        
        # 0-1の範囲にクリップ
        xmin = max(0.0, min(1.0, xmin))
        ymin = max(0.0, min(1.0, ymin))
        xmax = max(0.0, min(1.0, xmax))
        ymax = max(0.0, min(1.0, ymax))
        
        # YOLO形式に変換（center_x, center_y, width, height）
        center_x = (xmin + xmax) / 2.0
        center_y = (ymin + ymax) / 2.0
        width = xmax - xmin
        height = ymax - ymin
        
        # 0-1の範囲にクリップ
        center_x = max(0.0, min(1.0, center_x))
        center_y = max(0.0, min(1.0, center_y))
        width = max(0.0, min(1.0, width))
        height = max(0.0, min(1.0, height))
        
        # はみ出しチェック（x_center±w/2, y_center±h/2 が 0-1 内）
        if center_x - width/2 < 0 or center_x + width/2 > 1 or \
           center_y - height/2 < 0 or center_y + height/2 > 1:
            # クリップして再計算
            xmin_clipped = max(0.0, center_x - width/2)
            xmax_clipped = min(1.0, center_x + width/2)
            ymin_clipped = max(0.0, center_y - height/2)
            ymax_clipped = min(1.0, center_y + height/2)
            
            center_x = (xmin_clipped + xmax_clipped) / 2.0
            center_y = (ymin_clipped + ymax_clipped) / 2.0
            width = xmax_clipped - xmin_clipped
            height = ymax_clipped - ymin_clipped
        
        # ゼロ面積チェック
        if width <= 0 or height <= 0:
            skipped_invalid_bbox += 1
            if matched_function:
                i += matched_length
            else:
                i += 1
            continue
        
        class_ids.append(new_class_id)
        bboxes_yolo.append((center_x, center_y, width, height))
        stats[class_char] += 1
        
        if matched_function:
            i += matched_length
        else:
            i += 1
    # 対象クラスが1つもない場合はスキップ
    if len(class_ids) == 0:
        skipped_no_target += 1
        continue
    
    # 処理済みサンプルに追加
    processed_samples[uuid] = {
        'class_ids': class_ids,
        'bboxes_yolo': bboxes_yolo,
        'image_path': image_path
    }

print(f"\n✓ クラス抽出・正規化完了")
print(f"  処理済みサンプル: {len(processed_samples)} 件")
print(f"  スキップ（対象クラスなし）: {skipped_no_target} 件")
print(f"  スキップ（無効bbox）: {skipped_invalid_bbox} 件")
print(f"  スキップ（画像なし）: {skipped_no_image} 件")

print(f"\nクラス別インスタンス数:")
for cls in TARGET_CLASSES:
    count = stats[cls]
    print(f"  {cls:>3}: {count:>8} インスタンス")

total_instances = sum(stats.values())
print(f"\n総インスタンス数: {total_instances}")


## セル7: train/val split


In [ ]:
# ============================================
# train/val split
# ============================================

import random
from collections import Counter

print("=" * 80)
print("train/val split")
print("=" * 80)

# シードを設定
random.seed(SEED)

# UUIDのリストを取得してシャッフル
uuids = list(processed_samples.keys())
random.shuffle(uuids)

# 分割
split_idx = int(len(uuids) * (1 - VAL_RATIO))
train_uuids = uuids[:split_idx]
val_uuids = uuids[split_idx:]

print(f"\n分割結果:")
print(f"  Train: {len(train_uuids)} 件 ({len(train_uuids)/len(uuids)*100:.1f}%)")
print(f"  Val: {len(val_uuids)} 件 ({len(val_uuids)/len(uuids)*100:.1f}%)")
print(f"  総数: {len(uuids)} 件")

# クラス分布の確認（簡易サマリ）
train_class_counts = Counter()
val_class_counts = Counter()

for uuid in train_uuids:
    for class_id in processed_samples[uuid]['class_ids']:
        train_class_counts[class_id] += 1

for uuid in val_uuids:
    for class_id in processed_samples[uuid]['class_ids']:
        val_class_counts[class_id] += 1

print(f"\nクラス分布（簡易サマリ）:")
print(f"  Train 総インスタンス: {sum(train_class_counts.values())}")
print(f"  Val 総インスタンス: {sum(val_class_counts.values())}")

print(f"\n✓ train/val split完了")


## セル8: YOLOフォルダ出力


In [ ]:
# ============================================
# YOLOフォルダ出力
# ============================================

import shutil
import yaml

print("=" * 80)
print("YOLOフォルダ出力")
print("=" * 80)

# 出力ディレクトリの準備
if CLEAN_OUT_DIR and OUT_DIR.exists():
    print(f"🗑️  既存のOUT_DIRを削除: {OUT_DIR}")
    shutil.rmtree(OUT_DIR)

OUT_DIR.mkdir(parents=True, exist_ok=True)

# ディレクトリ構造を作成
train_images_dir = OUT_DIR / "train" / "images"
train_labels_dir = OUT_DIR / "train" / "labels"
val_images_dir = OUT_DIR / "val" / "images"
val_labels_dir = OUT_DIR / "val" / "labels"

for d in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
    d.mkdir(parents=True, exist_ok=True)

print(f"\n出力先: {OUT_DIR}")

# trainデータを出力
print(f"\n📁 trainデータを出力中...")
train_mapping = {}

for uuid in tqdm(train_uuids, desc="train"):
    sample = processed_samples[uuid]
    image_path = sample['image_path']
    
    # 画像をコピー（拡張子を保持）
    ext = image_path.suffix.lower()
    if ext not in ['.png', '.jpg', '.jpeg']:
        ext = '.png'
    
    target_image_path = train_images_dir / f"{uuid}{ext}"
    if not target_image_path.exists():
        shutil.copy2(image_path, target_image_path)
    
    # ラベルファイルを出力
    label_path = train_labels_dir / f"{uuid}.txt"
    with open(label_path, 'w', encoding='utf-8') as f:
        for class_id, (cx, cy, w, h) in zip(sample['class_ids'], sample['bboxes_yolo']):
            f.write(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")
    
    train_mapping[uuid] = str(target_image_path)

# valデータを出力
print(f"\n📁 valデータを出力中...")
val_mapping = {}

for uuid in tqdm(val_uuids, desc="val"):
    sample = processed_samples[uuid]
    image_path = sample['image_path']
    
    # 画像をコピー
    ext = image_path.suffix.lower()
    if ext not in ['.png', '.jpg', '.jpeg']:
        ext = '.png'
    
    target_image_path = val_images_dir / f"{uuid}{ext}"
    if not target_image_path.exists():
        shutil.copy2(image_path, target_image_path)
    
    # ラベルファイルを出力
    label_path = val_labels_dir / f"{uuid}.txt"
    with open(label_path, 'w', encoding='utf-8') as f:
        for class_id, (cx, cy, w, h) in zip(sample['class_ids'], sample['bboxes_yolo']):
            f.write(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")
    
    val_mapping[uuid] = str(target_image_path)

# mapping.jsonを生成
mapping = {
    'train': train_mapping,
    'val': val_mapping
}

mapping_file = OUT_DIR / "mapping.json"
with open(mapping_file, 'w', encoding='utf-8') as f:
    json.dump(mapping, f, ensure_ascii=False, indent=2)

print(f"\n✓ mapping.jsonを生成: {mapping_file}")

# data.yamlを生成（YOLOが読む設定ファイル）
data_yaml = {
    'path': str(OUT_DIR),
    'train': 'train/images',
    'val': 'val/images',
    'nc': len(TARGET_CLASSES),
    'names': {idx: cls for idx, cls in enumerate(TARGET_CLASSES)}
}

data_yaml_file = OUT_DIR / "data.yaml"
with open(data_yaml_file, 'w', encoding='utf-8') as f:
    yaml.dump(data_yaml, f, default_flow_style=False, allow_unicode=True)

print(f"\n✓ data.yamlを生成: {data_yaml_file}")

# 件数チェック
train_images = list(train_images_dir.glob("*"))
train_labels = list(train_labels_dir.glob("*.txt"))
val_images = list(val_images_dir.glob("*"))
val_labels = list(val_labels_dir.glob("*.txt"))

print(f"\n出力結果:")
print(f"  train/images: {len(train_images)} 件")
print(f"  train/labels: {len(train_labels)} 件")
print(f"  val/images: {len(val_images)} 件")
print(f"  val/labels: {len(val_labels)} 件")

# 1対1対応の確認
train_image_uuids = {f.stem for f in train_images}
train_label_uuids = {f.stem for f in train_labels}
val_image_uuids = {f.stem for f in val_images}
val_label_uuids = {f.stem for f in val_labels}

train_matched = train_image_uuids & train_label_uuids
val_matched = val_image_uuids & val_label_uuids

print(f"\n1対1対応確認:")
print(f"  train: {len(train_matched)}/{len(train_images)} ペア")
print(f"  val: {len(val_matched)}/{len(val_images)} ペア")

if len(train_matched) == len(train_images) == len(train_labels) and \
   len(val_matched) == len(val_images) == len(val_labels):
    print(f"\n✓ 完璧！全ての画像とラベルが1対1で対応しています。")
else:
    print(f"\n⚠️  警告: 一部のファイルが対応していません")

print(f"\n✓ YOLOフォルダ出力完了")


## セル9: 検証（重要）


In [ ]:
# ============================================
# 検証（重要）
# ============================================

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import random

print("=" * 80)
print("検証")
print("=" * 80)

# 件数チェック
print(f"\n件数チェック:")
print(f"  train/images: {len(train_images)} 件")
print(f"  train/labels: {len(train_labels)} 件")
print(f"  val/images: {len(val_images)} 件")
print(f"  val/labels: {len(val_labels)} 件")

# ランダムにサンプルを選んで表示
num_samples = 5
sample_uuids = random.sample(list(train_matched), min(num_samples, len(train_matched)))

print(f"\nサンプル表示（{num_samples}件）:")

fig, axes = plt.subplots(num_samples, 1, figsize=(12, 4 * num_samples))
if num_samples == 1:
    axes = [axes]

for idx, uuid in enumerate(sample_uuids):
    # 画像を読み込み
    image_files = list(train_images_dir.glob(f"{uuid}.*"))
    if len(image_files) == 0:
        continue
    
    image_path = image_files[0]
    img = Image.open(image_path)
    img_width, img_height = img.size
    
    # ラベルを読み込み
    label_path = train_labels_dir / f"{uuid}.txt"
    labels = []
    if label_path.exists():
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    center_x = float(parts[1])
                    center_y = float(parts[2])
                    width = float(parts[3])
                    height = float(parts[4])
                    
                    # YOLO形式からピクセル座標に変換
                    x_center = center_x * img_width
                    y_center = center_y * img_height
                    bbox_width = width * img_width
                    bbox_height = height * img_height
                    
                    xmin = x_center - bbox_width / 2
                    ymin = y_center - bbox_height / 2
                    xmax = x_center + bbox_width / 2
                    ymax = y_center + bbox_height / 2
                    
                    class_name = TARGET_CLASSES[class_id] if class_id < len(TARGET_CLASSES) else 'unknown'
                    labels.append({
                        'class_id': class_id,
                        'class_name': class_name,
                        'bbox': (xmin, ymin, xmax, ymax)
                    })
    
    # 画像を表示
    ax = axes[idx]
    ax.imshow(img)
    ax.axis('off')
    
    # バウンディングボックスを描画
    colors = plt.cm.tab20(range(len(TARGET_CLASSES)))
    for label in labels:
        class_id = label['class_id']
        class_name = label['class_name']
        xmin, ymin, xmax, ymax = label['bbox']
        
        # バウンディングボックス
        rect = patches.Rectangle(
            (xmin, ymin), xmax - xmin, ymax - ymin,
            linewidth=2, edgecolor=colors[class_id % len(colors)],
            facecolor='none'
        )
        ax.add_patch(rect)
        
        # クラス名を表示
        ax.text(
            xmin, ymin - 5, class_name,
            color=colors[class_id % len(colors)],
            fontsize=10, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7)
        )
    
    # タイトル
    title = f"UUID: {uuid[:8]}... | 検出数: {len(labels)}"
    if len(labels) > 0:
        classes_found = [l['class_name'] for l in labels]
        title += f" | クラス: {', '.join(set(classes_found))}"
    ax.set_title(title, fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# ラベルtxtの中身をチェック（最初の1件）
if len(sample_uuids) > 0:
    uuid = sample_uuids[0]
    label_path = train_labels_dir / f"{uuid}.txt"
    print(f"\nラベルファイルの中身確認（{uuid}.txt）:")
    with open(label_path, 'r') as f:
        lines = f.readlines()
        print(f"  行数: {len(lines)}")
        if len(lines) > 0:
            print(f"  最初の3行:")
            for i, line in enumerate(lines[:3]):
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    class_name = TARGET_CLASSES[class_id] if class_id < len(TARGET_CLASSES) else 'unknown'
                    print(f"    {i+1}: class={class_name}({class_id}) cx={parts[1]} cy={parts[2]} w={parts[3]} h={parts[4]}")

print(f"\n✓ 検証完了")


## セル10: manifest.json生成（再現性の核）


In [ ]:
# ============================================
# manifest.json生成（再現性の核）
# ============================================

import hashlib
import subprocess
from datetime import datetime

print("=" * 80)
print("manifest.json生成")
print("=" * 80)

# Git commit hashを取得（可能なら）
git_commit_hash = None
try:
    if repo_path.exists():
        result = subprocess.run(
            ["git", "-C", str(repo_path), "rev-parse", "HEAD"],
            capture_output=True, text=True, check=True
        )
        git_commit_hash = result.stdout.strip()
except Exception:
    pass

# manifest.jsonを生成（tarファイルはまだ作成されていないため、tar_filesは空）
manifest = {
    'version': '1.0',
    'created_at': datetime.now().isoformat(),
    'kaggle_dataset': KAGGLE_DATASET if RAW_ZIP_PATH is None else None,
    'raw_zip_path': str(RAW_ZIP_PATH) if RAW_ZIP_PATH else None,
    'target_classes': TARGET_CLASSES,
    'num_classes': len(TARGET_CLASSES),
    'split': {
        'seed': SEED,
        'val_ratio': VAL_RATIO,
        'train_count': len(train_uuids),
        'val_count': len(val_uuids)
    },
    'statistics': {
        'train_images': len(train_images),
        'train_labels': len(train_labels),
        'val_images': len(val_images),
        'val_labels': len(val_labels)
    },
    'git_commit_hash': git_commit_hash,
    'tar_files': {},  # tarファイルは後で作成されるため、ここでは空
    's3': {
        'bucket': S3_BUCKET if UPLOAD_TO_S3 else None,
        'prefix': S3_PREFIX if UPLOAD_TO_S3 else None,
        'uploaded': False  # まだアップロードされていない
    }
}

manifest_file = OUT_DIR / "manifest.json"
with open(manifest_file, 'w', encoding='utf-8') as f:
    json.dump(manifest, f, ensure_ascii=False, indent=2)

print(f"\n✓ manifest.jsonを生成: {manifest_file}")
print(f"\nmanifest.jsonの内容:")
print(json.dumps(manifest, ensure_ascii=False, indent=2))

print(f"\n✓ manifest.json生成完了")


## セル11: tar化（S3保管用）


In [ ]:
# ============================================
# tar化（S3保管用）
# ============================================

import tarfile
import time

print("=" * 80)
print("tar化（S3保管用）")
print("=" * 80)

# tarファイルの出力先
tar_dir = OUT_DIR.parent / "yolo_tars"
tar_dir.mkdir(parents=True, exist_ok=True)

print(f"\n出力先: {tar_dir}")

# tarファイルのパス
train_images_tar = tar_dir / "train_images.tar"
train_labels_tar = tar_dir / "train_labels.tar"
val_images_tar = tar_dir / "val_images.tar"
val_labels_tar = tar_dir / "val_labels.tar"
meta_tar = tar_dir / "meta.tar"

def create_tar(source_dir, tar_path, desc):
    """tarファイルを作成"""
    start_time = time.time()
    print(f"\n📦 {desc}を作成中...")
    
    with tarfile.open(tar_path, 'w') as tar:
        # ディレクトリ内の全ファイルを追加
        for file_path in tqdm(list(source_dir.rglob('*')), desc=desc):
            if file_path.is_file():
                # 相対パスで追加
                arcname = file_path.relative_to(source_dir)
                tar.add(file_path, arcname=arcname)
    
    elapsed = time.time() - start_time
    size_mb = tar_path.stat().st_size / (1024 * 1024)
    print(f"  ✓ 完了: {tar_path.name} ({size_mb:.2f} MB, {elapsed:.1f}秒)")
    return size_mb

# train_images.tar
if not train_images_tar.exists():
    create_tar(train_images_dir, train_images_tar, "train_images.tar")
else:
    size_mb = train_images_tar.stat().st_size / (1024 * 1024)
    print(f"\n✓ train_images.tar は既に存在: {size_mb:.2f} MB")

# train_labels.tar
if not train_labels_tar.exists():
    create_tar(train_labels_dir, train_labels_tar, "train_labels.tar")
else:
    size_mb = train_labels_tar.stat().st_size / (1024 * 1024)
    print(f"\n✓ train_labels.tar は既に存在: {size_mb:.2f} MB")

# val_images.tar
if not val_images_tar.exists():
    create_tar(val_images_dir, val_images_tar, "val_images.tar")
else:
    size_mb = val_images_tar.stat().st_size / (1024 * 1024)
    print(f"\n✓ val_images.tar は既に存在: {size_mb:.2f} MB")

# val_labels.tar
if not val_labels_tar.exists():
    create_tar(val_labels_dir, val_labels_tar, "val_labels.tar")
else:
    size_mb = val_labels_tar.stat().st_size / (1024 * 1024)
    print(f"\n✓ val_labels.tar は既に存在: {size_mb:.2f} MB")

# meta.tar（mapping.json, data.yaml, manifest.json）
manifest_file = OUT_DIR / "manifest.json"
if not manifest_file.exists():
    raise FileNotFoundError(f"manifest.jsonが見つかりません: {manifest_file}\nセル10（manifest.json生成）を先に実行してください。")

meta_files = [mapping_file, data_yaml_file, manifest_file]

if not meta_tar.exists():
    start_time = time.time()
    print(f"\n📦 meta.tarを作成中...")
    
    with tarfile.open(meta_tar, 'w') as tar:
        for file_path in meta_files:
            if file_path.exists():
                arcname = file_path.name
                tar.add(file_path, arcname=arcname)
    
    elapsed = time.time() - start_time
    size_mb = meta_tar.stat().st_size / (1024 * 1024)
    print(f"  ✓ 完了: {meta_tar.name} ({size_mb:.2f} MB, {elapsed:.1f}秒)")
else:
    size_mb = meta_tar.stat().st_size / (1024 * 1024)
    print(f"\n✓ meta.tar は既に存在: {size_mb:.2f} MB")

# 合計サイズ
total_size = sum(f.stat().st_size for f in [train_images_tar, train_labels_tar, val_images_tar, val_labels_tar, meta_tar] if f.exists())
total_size_mb = total_size / (1024 * 1024)
print(f"\n✓ tar化完了")
print(f"  合計サイズ: {total_size_mb:.2f} MB")
print(f"  出力先: {tar_dir}")


## セル12: S3へアップロード（任意）


In [ ]:
# ============================================
# S3へアップロード（任意）
# ============================================

import subprocess
import time

if not UPLOAD_TO_S3:
    print("⚠️  UPLOAD_TO_S3=False のため、S3アップロードをスキップします")
    print(f"   アップロードする場合は、セル1の設定で UPLOAD_TO_S3=True にしてください")
else:
    print("=" * 80)
    print("S3へアップロード")
    print("=" * 80)
    
    # AWS認証情報の確認
    try:
        result = subprocess.run(["aws", "sts", "get-caller-identity"], 
                              capture_output=True, text=True, check=True)
        print(f"\n✓ AWS認証情報を確認")
    except subprocess.CalledProcessError:
        print("\n❌ エラー: AWS認証情報が設定されていません")
        print("   aws configure を実行するか、環境変数を設定してください")
        raise
    
    tar_files = [
        (train_images_tar, "train_images.tar"),
        (train_labels_tar, "train_labels.tar"),
        (val_images_tar, "val_images.tar"),
        (val_labels_tar, "val_labels.tar"),
        (meta_tar, "meta.tar")
    ]
    
    uploaded = []
    failed = []
    
    for tar_path, tar_name in tar_files:
        if not tar_path.exists():
            print(f"\n⚠️  スキップ: {tar_name} が見つかりません")
            continue
        
        s3_path = f"s3://{S3_BUCKET}/{S3_PREFIX}/{tar_name}"
        
        print(f"\n📤 {tar_name} をアップロード中...")
        print(f"   S3パス: {s3_path}")
        
        max_retries = 3
        retry_count = 0
        success = False
        
        while retry_count < max_retries and not success:
            try:
                start_time = time.time()
                result = subprocess.run(
                    ["aws", "s3", "cp", str(tar_path), s3_path],
                    capture_output=True, text=True, check=True
                )
                elapsed = time.time() - start_time
                size_mb = tar_path.stat().st_size / (1024 * 1024)
                print(f"  ✓ 完了: {size_mb:.2f} MB, {elapsed:.1f}秒")
                uploaded.append(tar_name)
                success = True
            except subprocess.CalledProcessError as e:
                retry_count += 1
                if retry_count < max_retries:
                    print(f"  ⚠️  リトライ {retry_count}/{max_retries}...")
                    time.sleep(2)
                else:
                    print(f"  ❌ 失敗: {e.stderr}")
                    failed.append(tar_name)
    
    print(f"\n" + "=" * 80)
    print("アップロード結果")
    print("=" * 80)
    print(f"  成功: {len(uploaded)} 件")
    if uploaded:
        for name in uploaded:
            print(f"    ✓ {name}")
    
    print(f"  失敗: {len(failed)} 件")
    if failed:
        for name in failed:
            print(f"    ❌ {name}")
    
    if len(uploaded) == len(tar_files):
        print(f"\n✓ 全てのファイルのアップロードが完了しました")
        print(f"  S3パス: s3://{S3_BUCKET}/{S3_PREFIX}/")
    else:
        print(f"\n⚠️  一部のファイルのアップロードに失敗しました")
        print(f"  失敗したファイルは手動でアップロードしてください")
